In [ ]:
#install needed packages
!pip install snorkel
!pip install textblob
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.6 MB/s eta 0:00:00


In [ ]:
# pandas and numpy
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

#import libraries and modules
from google.colab import files
import io

#import warnings
#warnings.filterwarnings('ignore')

#Snorkel
from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function

# API
import requests
import json

# sentiment analysis
import spacy
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# NLTK
import nltk
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import string
from string import punctuation
from nltk.corpus import stopwords

import nltk.tokenize
punc = string.punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer  # lemmatization
nltk.download('wordnet')

from statistics import mean
from heapq import nlargest

# miscellaneous
import time as timer
from datetime import datetime, date, time
from tqdm import tqdm
from collections import Counter
import pickle
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt

#Supervised learning
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

##Deep learning libraries and APIs
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load and prepare data

In [ ]:
# WITH TEST DATA

# Importing the dataset
df_1 = pd.read_csv("/content/drive/MyDrive/info/test_data/articles1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/info/test_data/articles2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/info/test_data/articles3.csv")

In [ ]:
# Checking if the columns are same or not
df_1.columns == df_2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [ ]:
# Checking if the columns are same or not
df_2.columns == df_3.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [ ]:
# Making one Dataframe by appending all of them for the further process
d = [df_1, df_2, df_3]
df = pd.concat(d, keys = ['x', 'y', 'z'])
df.rename(columns = {'content' : 'article'}, inplace = True);

In [ ]:
df.head()

Unnamed: 0     id                                              title  \
x 0           0  17283  House Republicans Fret About Winning Their Hea...   
  1           1  17284  Rift Between Officers and Residents as Killing...   
  2           2  17285  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3           3  17286  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4           4  17287  Kim Jong-un Says North Korea Is Preparing to T...   

        publication                         author        date    year  month  \
x 0  New York Times                     Carl Hulse  2016-12-31  2016.0   12.0   
  1  New York Times  Benjamin Mueller and Al Baker  2017-06-19  2017.0    6.0   
  2  New York Times                   Margalit Fox  2017-01-06  2017.0    1.0   
  3  New York Times               William McDonald  2017-04-10  2017.0    4.0   
  4  New York Times                  Choe Sang-Hun  2017-01-02  2017.0    1.0   

     url                                            article  
x 0  NaN  WASHINGTON  —   Congressional Republicans have...  
  1  NaN  After the bullet shells get counted, the blood...  
  2  NaN  When Walt Disney’s “Bambi” opened in 1942, cri...  
  3  NaN  Death may be the great equalizer, but it isn’t...  
  4  NaN  SEOUL, South Korea  —   North Korea’s leader, ...

In [ ]:
# Dropping the unnecessary columns
df.drop(columns = ['Unnamed: 0', 'id', 'publication', 'author', 'date', 'year', 'month', 'url'], inplace = True)
df.head()

title  \
x 0  House Republicans Fret About Winning Their Hea...   
  1  Rift Between Officers and Residents as Killing...   
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4  Kim Jong-un Says North Korea Is Preparing to T...   

                                               article  
x 0  WASHINGTON  —   Congressional Republicans have...  
  1  After the bullet shells get counted, the blood...  
  2  When Walt Disney’s “Bambi” opened in 1942, cri...  
  3  Death may be the great equalizer, but it isn’t...  
  4  SEOUL, South Korea  —   North Korea’s leader, ...

In [ ]:
df['title'] = df['title'].astype(str)
df['article'] = df['article'].astype(str)
#check the data info
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 142570 entries, ('x', 0) to ('z', 42570)
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   title    142570 non-null  object
 1   article  142570 non-null  object
dtypes: object(2)
memory usage: 4.2+ MB


In [ ]:
df_article = df[['article']].copy()
df_title = df[['title']].copy()

df_title.head()

title
x 0  House Republicans Fret About Winning Their Hea...
  1  Rift Between Officers and Residents as Killing...
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...
  4  Kim Jong-un Says North Korea Is Preparing to T...

In [ ]:
df_article.head()

article
x 0  WASHINGTON  —   Congressional Republicans have...
  1  After the bullet shells get counted, the blood...
  2  When Walt Disney’s “Bambi” opened in 1942, cri...
  3  Death may be the great equalizer, but it isn’t...
  4  SEOUL, South Korea  —   North Korea’s leader, ...

# Text Processing

In [ ]:
title_tt = df_title.copy()
article_tt = df_article.copy()

In [ ]:
def pre_process(df):

    publishers = ['The New York Times', 'Breitbart', 'CNN', 'Business Insider', 'Fox News', 'Talking Points Memo', 'Buzzfeed News', 'National Review', 'New York Post', 'The Guardian', 'NPR', 'Reuters', 'Vox', 'Washington Post', 'Associated Press']

    text = df
    #print(text)

    for a in publishers:
      if a in df:
        text = df.replace(a, '')

    #print(text)


    # take care of punction
    text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", text) # when at the beginning of a string, separate punctuation
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text) # when at the end of a string, separate punctuation

    # remove any numbers
    numbers = r'\d+'
    text = re.sub(pattern=numbers, repl=" ", string=text)

    # split the string into separate tokens
    tokens = re.split(r"\s+",text)


    # normalise all words into lowercase
    final = [t.lower() for t in tokens]


    tokens = []

    # remove any strings signalising end of line
    for i in range(len(final)):
        if final[i] != '-' and final[i] != '—' and final[i] != '“' and final[i] != '”':
            tokens.append(final[i])

    # remove any unncessary punctuation connected to words
    x = '[{}]'.format(re.escape(string.punctuation)+'\…').replace("...", "").replace("\-", "")
    pattern = re.compile(x)
    tokens = [f for f in filter(None, [pattern.sub('', token) for token in tokens])]

    # remove stop words
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwords]

    # apply lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(token) for token in tokens]


    # return final list of tokens
    return tokens

In [ ]:
# TEST
text_test = "I have the — high ground ” , “ Anakin! - The New York Times"
tokens_test = pre_process(text_test)
print(tokens_test)

['high', 'ground', 'anakin']


In [ ]:
tokenised = ['']*142570
title_tt['tokenised'] = tokenised

for index, row in title_tt.iterrows():
  token = []
  token = pre_process(row['title'])

  # Using .join() to Convert a List to a String
  conv_token = ' '.join(token)

  title_tt.loc[index, 'tokenised'] = conv_token

title_tt.head()

title  \
x 0  House Republicans Fret About Winning Their Hea...   
  1  Rift Between Officers and Residents as Killing...   
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4  Kim Jong-un Says North Korea Is Preparing to T...   

                                             tokenised  
x 0     house republican fret winning health care suit  
  1  rift officer resident killing persist south bronx  
  2  tyrus wong ‘bambi’ artist thwarted racial bias dy  
  3                   among death heavy toll pop music  
  4  kim jong-un say north korea preparing test lon...

In [ ]:
tokenised = ['']*142570
article_tt['tokenised'] = tokenised

for index, row in article_tt.iterrows():
  token = []
  token = pre_process(row['article'])

  # Using .join() to Convert a List to a String
  conv_token = ' '.join(token)

  article_tt.loc[index, 'tokenised'] = conv_token

article_tt.head()

article  \
x 0  WASHINGTON  —   Congressional Republicans have...   
  1  After the bullet shells get counted, the blood...   
  2  When Walt Disney’s “Bambi” opened in 1942, cri...   
  3  Death may be the great equalizer, but it isn’t...   
  4  SEOUL, South Korea  —   North Korea’s leader, ...   

                                             tokenised  
x 0  washington congressional republican new fear c...  
  1  bullet shell get counted blood dry votive cand...  
  2  walt disney’s bambi opened critic praised spar...  
  3  death may great equalizer isn’t necessarily ev...  
  4  seoul south korea north korea’s leader kim sai...

In [ ]:
# save values for future uses
article_tt.to_csv("/content/drive/MyDrive/info/article_tt.csv", sep='\t')
title_tt.to_csv("/content/drive/MyDrive/info/title_tt.csv", sep='\t')

In [ ]:
# in case data needs to be uploaded
article_tt = pd.read_csv("/content/drive/MyDrive/info/article_tt.csv", sep='\t', lineterminator='\n')
title_tt = pd.read_csv("/content/drive/MyDrive/info/title_tt.csv", sep='\t', lineterminator='\n')

article_tt.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)
title_tt.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)

article_tt.head()
title_tt.head()

,title,tokenised
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...


# Create labels with Snorkel

In [ ]:
#title_sn = df_title.copy()
#article_sn = df_article.copy()

title_sn = title_tt.copy()
article_sn = article_tt.copy()

In [ ]:
#define constants to represent the class labels :positive, negative, and abstain
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

#define function which looks into the input words to represent a proper label
def keyword_lookup(x, keywords, label):
    if any(word in x.tokenised for word in keywords):
        return label
    return ABSTAIN

#define function which assigns a correct label
def make_keyword_lf(keywords, label=POSITIVE):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label))

#resource: https://www.snorkel.org/use-cases/01-spam-tutorial#3-writing-more-labeling-functions
#these two lists can be further extended

"""positive news might contain the following words' """
keyword_positive = make_keyword_lf(keywords=['boosts', 'great', 'develops', 'promising', 'ambitious', 'delighted', 'record', 'win', 'breakthrough', 'recover', 'achievement', 'peace', 'party', 'hope', 'flourish', 'respect', 'partnership', 'champion', 'positive', 'happy', 'bright', 'confident', 'encouraged', 'perfect', 'complete', 'assured' ])
"""negative news might contain the following words"""
keyword_negative = make_keyword_lf(keywords=['war','solidiers', 'turmoil', 'injur','trouble', 'aggressive', 'killed', 'coup', 'evasion', 'strike', 'troops', 'dismisses', 'attacks', 'defeat', 'damage', 'dishonest', 'dead', 'fear', 'foul', 'fails', 'hostile', 'cuts', 'accusations', 'victims',  'death', 'unrest', 'fraud', 'dispute', 'destruction', 'battle', 'unhappy', 'bad', 'alarming', 'angry', 'anxious', 'dirty', 'pain', 'poison', 'unfair', 'unhealthy'
                                              ], label=NEGATIVE)

#set up a preprocessor function to determine polarity & subjectivity using textlob pretrained classifier
@preprocessor(memoize=True)
def textblob_sentiment(x):

    scores = TextBlob(x.tokenised)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

#find polarity
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return POSITIVE if x.polarity > 0.6 else ABSTAIN

#find subjectivity
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return POSITIVE if x.subjectivity >= 0.5 else ABSTAIN

In [ ]:
#combine all the labeling functions
lfs = [keyword_positive, keyword_negative, textblob_polarity, textblob_subjectivity ]

#apply the lfs on the dataframe
applier = PandasLFApplier(lfs=lfs)

L_snorkel_t = applier.apply(df=title_sn)
L_snorkel_a = applier.apply(df=article_sn)

#apply the label model
label_model_t = LabelModel(cardinality=2, verbose=True)
label_model_a = LabelModel(cardinality=2, verbose=True)

#fit on the data
label_model_t.fit(L_snorkel_t)
label_model_a.fit(L_snorkel_a)

#predict and create the labels
title_sn["label"] = label_model_t.predict(L=L_snorkel_t)
article_sn["label"] = label_model_a.predict(L=L_snorkel_a)

#Filtering out unlabeled data points
#title_sn_fin = title_sn.loc[title_sn.label.isin([0,1]), :]
#article_sn_fin = article_sn.loc[article_sn.label.isin([0,1]), :]

#find the label counts
print(title_sn['label'].value_counts())
print(article_sn['label'].value_counts())
#print(title_sn_fin['label'].value_counts())
#print(article_sn_fin['label'].value_counts())

100%|██████████| 100/100 [00:00<00:00, 1042.89epoch/s]


-1    92338
 1    39753
 0    10479
Name: label, dtype: int64
 1    119065
 0     15068
-1      8437
Name: label, dtype: int64


In [ ]:
#headline is positive (1) or negative (0)
title_sn.head()

title  \
x 0  House Republicans Fret About Winning Their Hea...   
  1  Rift Between Officers and Residents as Killing...   
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4  Kim Jong-un Says North Korea Is Preparing to T...   

                                             tokenised  label  
x 0     house republican fret winning health care suit      1  
  1  rift officer resident killing persist south bronx     -1  
  2  tyrus wong ‘bambi’ artist thwarted racial bias dy      0  
  3                   among death heavy toll pop music      1  
  4  kim jong-un say north korea preparing test lon...     -1

In [ ]:
article_sn.head()

article  \
x 0  WASHINGTON  —   Congressional Republicans have...   
  1  After the bullet shells get counted, the blood...   
  2  When Walt Disney’s “Bambi” opened in 1942, cri...   
  3  Death may be the great equalizer, but it isn’t...   
  4  SEOUL, South Korea  —   North Korea’s leader, ...   

                                             tokenised  label  
x 0  washington congressional republican new fear c...      1  
  1  bullet shell get counted blood dry votive cand...      1  
  2  walt disney’s bambi opened critic praised spar...      1  
  3  death may great equalizer isn’t necessarily ev...      1  
  4  seoul south korea north korea’s leader kim sai...      1

In [ ]:
# save values for future uses
article_sn.to_csv("/content/drive/MyDrive/info/article_sn.csv", sep='\t')
title_sn.to_csv("/content/drive/MyDrive/info/title_sn.csv", sep='\t')

In [ ]:
# in case data needs to be uploaded
article_sn = pd.read_csv("/content/drive/MyDrive/info/article_sn.csv", sep='\t', lineterminator='\n')
title_sn = pd.read_csv("/content/drive/MyDrive/info/title_sn.csv", sep='\t', lineterminator='\n')

article_sn.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)
title_sn.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)

article_sn.head()
title_sn.head()

,title,tokenised,label
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit,1
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx,-1
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy,0
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music,1
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...,-1


In [ ]:
a_sn_labels = article_sn['label']
t_sn_labels = title_sn['label']

a_sn_labels.to_csv("/content/drive/MyDrive/info/a_sn_labels.csv", sep='\t')
t_sn_labels.to_csv("/content/drive/MyDrive/info/t_sn_labels.csv", sep='\t')

# Manually labeling using Dictionaries

In [ ]:
#title_di = df_title.copy()
#article_di = df_article.copy()

title_di = title_tt.copy()
article_di = article_tt.copy()

In [ ]:
!cp /content/drive/MyDrive/info/dictionary.py .
import dictionary

In [ ]:
# from https://inquirer.sites.fas.harvard.edu/spreadsheet_guide.htm
gi_dict_path = "/content/drive/MyDrive/info/gi_dictionary.csv"

gi_df = pd.read_csv(gi_dict_path)

gi_df.head()

<ipython-input-27-6d358ba55861>:4: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  gi_df = pd.read_csv(gi_dict_path)


,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
0,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
1,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
2,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
3,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
4,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,NaN


In [ ]:
gi_pos_tmp = gi_df[['Entry', 'Positiv']].copy()
gi_neg_tmp = gi_df[['Entry', 'Negativ']].copy()

mask_pos = gi_pos_tmp['Positiv'].isnull()
mask_neg = gi_neg_tmp['Negativ'].isnull()

gi_pos = gi_pos_tmp[~mask_pos]
gi_neg = gi_neg_tmp[~mask_neg]

gi_pos.head()

,Entry,Positiv
7,ABIDE,Positiv
8,ABILITY,Positiv
10,ABLE,Positiv
17,ABOUND,Positiv
40,ABSOLVE,Positiv


In [ ]:
gi_neg.head()

,Entry,Negativ
1,ABANDON,Negativ
2,ABANDONMENT,Negativ
3,ABATE,Negativ
5,ABDICATE,Negativ
6,ABHOR,Negativ


In [ ]:
import os

#import Load_MasterDictionary as MD ## LM Dictionary
file_path = ('/content/drive/MyDrive/info/Loughran-McDonald_MasterDictionary_1993-2021.csv')

md, header, sent_cat, sent_dict, stopwords, total_doc = dictionary.load_masterdictionary(file_path, get_other = True)

## Convert the LM Dictionary into positive/negative word lists
positive = []
negative = []
for sent, value in sent_dict.items():
    if sent == "negative" or sent == "uncertainty":
        for word in value.keys():
            negative.append(word.lower())
            #print(negative)
    elif sent == "positive" :
        for word in value.keys():
            positive.append(word.lower())
            #print(positive)

## GI Dictionary
gi_pos_list = list(gi_pos['Entry'])
gi_neg_list = list(gi_neg['Entry'])

print(gi_pos_list)
print(gi_neg_list)


['ABIDE', 'ABILITY', 'ABLE', 'ABOUND', 'ABSOLVE', 'ABSORBENT', 'ABSORPTION', 'ABUNDANCE', 'ABUNDANT', 'ACCEDE', 'ACCENTUATE', 'ACCEPT', 'ACCEPTABLE', 'ACCEPTANCE', 'ACCESSIBLE', 'ACCESSION', 'ACCLAIM', 'ACCLAMATION', 'ACCOLADE', 'ACCOMMODATE', 'ACCOMMODATION', 'ACCOMPANIMENT', 'ACCOMPLISH', 'ACCOMPLISHMENT', 'ACCORD#2', 'ACCORD#3', 'ACCORD#5', 'ACCORDANCE', 'ACCOUNTABLE', 'ACCRUE', 'ACCURACY', 'ACCURATE', 'ACCURATENESS', 'ACHIEVE', 'ACHIEVEMENT', 'ACKNOWLEDGEMENT', 'ACQUAINT', 'ACQUAINTANCE', 'ACQUIT', 'ACQUITTAL', 'ACTUAL#1', 'ACTUAL#2', 'ACTUALITY', 'ADAMANT', 'ADAPTABILITY', 'ADAPTABLE', 'ADAPTATION', 'ADAPTIVE', 'ADEPT', 'ADEPTNESS', 'ADEQUATE', 'ADHERENCE', 'ADHERENT', 'ADHESION', 'ADHESIVE', 'ADJUNCT', 'ADJUST#2', 'ADJUSTABLE', 'ADJUSTMENT', 'ADMIRABLE', 'ADMIRATION', 'ADMIRE', 'ADMIRER', 'ADMIT#1', 'ADMIT#2', 'ADMIT#3', 'ADMITTANCE', 'ADORABLE', 'ADORE', 'ADORN', 'ADORNMENT', 'ADROIT', 'ADROITLY', 'ADULATION', 'ADULT#2', 'ADVANCE#1', 'ADVANCE#2', 'ADVANCE#3', 'ADVANCEMENT', 'AD

In [ ]:
## Extra words defined by the user to be relevant in the analysis
positive_extra = [r"[bB]ull", "positive", "rise", "stock", "high", "up", "climb", "buy", "surge", "recover",
                  "open", "healthy", "dream", "together", "higher", "best",
                  "soar", "up", "blue", "chip", "life", "announcement", "designed", "prevent", "legalize", "yes", "finally"]
negative_extra = [r'[bB]ear', "negative", "drop", "fall", "low", "hacking", "freakout", "funeral", "terrorist",
                  "fight", "crisis", "killer", "taliban", "airstrikes", "hacker", "die", "warns", "assault",
                  "burst", "tight", "bubble", "crash", "down", "slide", "sell", "plunge", "dive",
                  "stagflation", "recession", "slow", "inflation", "blowhard", "down", "death", "sick", "killing", "toll", "missile", "bomb", "bias"]

## Combine all lists into a single list
positive += (positive_extra + gi_pos_list)
negative += (negative_extra + gi_neg_list)

## Remove duplicates and convert to a final list
positive_w = list(set(positive))
negative_w = list(set(negative))

positive_words = []
negative_words = []

for word in positive_w:
  token = pre_process(word)

  # Using .join() to Convert a List to a String
  conv_token = ' '.join(token)

  positive_words.append(conv_token)

for word in negative_w:
  token = pre_process(word)

  # Using .join() to Convert a List to a String
  conv_token = ' '.join(token)

  negative_words.append(conv_token)

print(positive_words)
print(negative_words)

['consent', 'simplify', 'adroit', 'genial', 'blossom', 'dependable', 'enhancement', 'uphold', 'worth', 'exceptional', 'admit', 'achievement', 'upturn', 'offer', 'empowers', 'embellish', 'fragrant', 'coexistence', 'thankful', 'respect', 'prospered', 'enrich', 'vanquish', 'ability', 'reinforcement', 'relevancy', 'renovate', 'designed', 'bestow', 'contribute', 'ideal', 'vibrancy', 'reward', 'allegiance', 'healthy', 'loyalty', 'delicate', 'complimentary', 'actual', 'affectionate', 'entrust', 'eager', 'sincere', 'excited', 'neat', 'advancement', 'outright', 'thrill', 'innovator', 'fit', 'fruitful', 'clearness', 'improvement', 'enhance', 'optimistic', 'enthusiasm', 'compliment', 'enjoyment', 'skillful', 'award', 'sensational', 'confederation', 'utilization', 'favorably', 'comprehension', 'inspirational', 'mature', 'trust', 'warm', 'credibility', 'establish', 'benefitting', 'astound', 'advance', 'reward', 'noiseless', 'superior', 'nobility', 'exclusively', 'cogent', 'heartily', 'paradise', 'a

In [ ]:
title_di.shape

(142570, 2)

In [ ]:
labels = []
count = 0

for index, row in title_di.iterrows():

    count += 1
    print(count)

    label_tmp = []
    r = row['tokenised'].split()

    for word in r:
      #print(word)

      if word in positive_words:
        label_tmp.append(1)

      elif word in negative_words:
        label_tmp.append(0)

      else:
        label_tmp.append(-1)

    if len(label_tmp) != 0:

      pos = label_tmp.count(1)
      neg = label_tmp.count(0)
      neu = label_tmp.count(-1)
      #print(label_tmp)

      total = len(label_tmp)
      pos_score = pos/total
      neg_score = neg/total
      neu_score = neu/total

      score = max([pos_score, neg_score])
      #print(score)

      if neg_score > pos_score:
          score = -score
          labels.append(score)
      else:
          labels.append(score)
    else:
      labels.append(0)

title_di['label'] = labels

title_di.head()

Streaming output truncated to the last 5000 lines.
137571
137572
137573
137574
137575
137576
137577
137578
137579
137580
137581
137582
137583
137584
137585
137586
137587
137588
137589
137590
137591
137592
137593
137594
137595
137596
137597
137598
137599
137600
137601
137602
137603
137604
137605
137606
137607
137608
137609
137610
137611
137612
137613
137614
137615
137616
137617
137618
137619
137620
137621
137622
137623
137624
137625
137626
137627
137628
137629
137630
137631
137632
137633
137634
137635
137636
137637
137638
137639
137640
137641
137642
137643
137644
137645
137646
137647
137648
137649
137650
137651
137652
137653
137654
137655
137656
137657
137658
137659
137660
137661
137662
137663
137664
137665
137666
137667
137668
137669
137670
137671
137672
137673
137674
137675
137676
137677
137678
137679
137680
137681
137682
137683
137684
137685
137686
137687
137688
137689
137690
137691
137692
137693
137694
137695
137696
137697
137698
137699
137700
137701
137702
137703
137704
137705
1377

title  \
x 0  House Republicans Fret About Winning Their Hea...   
  1  Rift Between Officers and Residents as Killing...   
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4  Kim Jong-un Says North Korea Is Preparing to T...   

                                             tokenised     label  
x 0     house republican fret winning health care suit  0.571429  
  1  rift officer resident killing persist south bronx -0.285714  
  2  tyrus wong ‘bambi’ artist thwarted racial bias dy -0.125000  
  3                   among death heavy toll pop music -0.333333  
  4  kim jong-un say north korea preparing test lon... -0.111111

In [ ]:
print(title_di['label'].value_counts())

 0.000000    30040
 0.142857     7458
-0.250000     6931
 0.125000     6836
 0.166667     6120
             ...  
-0.277778        1
 0.055556        1
 0.105263        1
-0.875000        1
-0.636364        1
Name: label, Length: 115, dtype: int64


In [ ]:
labels = []
count = 0

for index, row in article_di.iterrows():

    count += 1
    print(count)

    label_tmp = []
    r = row['tokenised'].split()

    for word in r:
      #print(word)

      if word in positive_words:
        label_tmp.append(1)

      elif word in negative_words:
        label_tmp.append(0)

      else:
        label_tmp.append(-1)

    if len(label_tmp) != 0:

      pos = label_tmp.count(1)
      neg = label_tmp.count(0)
      neu = label_tmp.count(-1)
      #print(label_tmp)

      total = len(label_tmp)
      pos_score = pos/total
      neg_score = neg/total
      neu_score = neu/total

      score = max([pos_score, neg_score])
      #print(score)

      if neg_score > pos_score:
          score = -score
          labels.append(score)
      else:
          labels.append(score)
    else:
      labels.append(0)

article_di['label'] = labels

article_di.head()

Streaming output truncated to the last 5000 lines.
137571
137572
137573
137574
137575
137576
137577
137578
137579
137580
137581
137582
137583
137584
137585
137586
137587
137588
137589
137590
137591
137592
137593
137594
137595
137596
137597
137598
137599
137600
137601
137602
137603
137604
137605
137606
137607
137608
137609
137610
137611
137612
137613
137614
137615
137616
137617
137618
137619
137620
137621
137622
137623
137624
137625
137626
137627
137628
137629
137630
137631
137632
137633
137634
137635
137636
137637
137638
137639
137640
137641
137642
137643
137644
137645
137646
137647
137648
137649
137650
137651
137652
137653
137654
137655
137656
137657
137658
137659
137660
137661
137662
137663
137664
137665
137666
137667
137668
137669
137670
137671
137672
137673
137674
137675
137676
137677
137678
137679
137680
137681
137682
137683
137684
137685
137686
137687
137688
137689
137690
137691
137692
137693
137694
137695
137696
137697
137698
137699
137700
137701
137702
137703
137704
137705
1377

article  \
x 0  WASHINGTON  —   Congressional Republicans have...   
  1  After the bullet shells get counted, the blood...   
  2  When Walt Disney’s “Bambi” opened in 1942, cri...   
  3  Death may be the great equalizer, but it isn’t...   
  4  SEOUL, South Korea  —   North Korea’s leader, ...   

                                             tokenised     label  
x 0  washington congressional republican new fear c...  0.152174  
  1  bullet shell get counted blood dry votive cand... -0.124178  
  2  walt disney’s bambi opened critic praised spar...  0.078089  
  3  death may great equalizer isn’t necessarily ev... -0.089478  
  4  seoul south korea north korea’s leader kim sai... -0.070588

In [ ]:
print(article_di['label'].value_counts())

 0.000000    448
 0.111111    442
-0.125000    407
 0.100000    406
-0.111111    388
            ... 
 0.122291      1
-0.137218      1
 0.086907      1
-0.105597      1
-0.074725      1
Name: label, Length: 36872, dtype: int64


In [ ]:
# save values for future uses
article_di.to_csv("/content/drive/MyDrive/info/article_di.csv", sep='\t')
title_di.to_csv("/content/drive/MyDrive/info/title_di.csv", sep='\t')

In [ ]:
# in case data needs to be uploaded
article_di = pd.read_csv("/content/drive/MyDrive/info/article_di.csv", sep='\t', lineterminator='\n')
title_di = pd.read_csv("/content/drive/MyDrive/info/title_di.csv", sep='\t', lineterminator='\n')

article_di.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)
title_di.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)

article_di.head()
title_di.head()

,title,tokenised,label
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit,0.571429
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx,-0.285714
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy,-0.125000
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music,-0.333333
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...,-0.111111


In [ ]:
a_di_labels = article_di['label']
t_di_labels = title_di['label']

a_di_labels.to_csv("/content/drive/MyDrive/info/a_di_labels.csv", sep='\t')
t_di_labels.to_csv("/content/drive/MyDrive/info/t_di_labels.csv", sep='\t')

# Labeling with TextBlob

In [ ]:
#title_tb = df_title.copy()
#article_tb = df_article.copy()

title_tb = title_tt.copy()
article_tb = article_tt.copy()

In [ ]:
polar = []
count = 0

for index, row in title_tb.iterrows():
    count += 1
    print(count)

    textBlob = TextBlob(row['tokenised'])
    polar.append(textBlob.polarity * textBlob.subjectivity)

title_tb['label'] = polar

title_tb.head()

Streaming output truncated to the last 5000 lines.
137571
137572
137573
137574
137575
137576
137577
137578
137579
137580
137581
137582
137583
137584
137585
137586
137587
137588
137589
137590
137591
137592
137593
137594
137595
137596
137597
137598
137599
137600
137601
137602
137603
137604
137605
137606
137607
137608
137609
137610
137611
137612
137613
137614
137615
137616
137617
137618
137619
137620
137621
137622
137623
137624
137625
137626
137627
137628
137629
137630
137631
137632
137633
137634
137635
137636
137637
137638
137639
137640
137641
137642
137643
137644
137645
137646
137647
137648
137649
137650
137651
137652
137653
137654
137655
137656
137657
137658
137659
137660
137661
137662
137663
137664
137665
137666
137667
137668
137669
137670
137671
137672
137673
137674
137675
137676
137677
137678
137679
137680
137681
137682
137683
137684
137685
137686
137687
137688
137689
137690
137691
137692
137693
137694
137695
137696
137697
137698
137699
137700
137701
137702
137703
137704
137705
1377

title  \
x 0  House Republicans Fret About Winning Their Hea...   
  1  Rift Between Officers and Residents as Killing...   
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4  Kim Jong-un Says North Korea Is Preparing to T...   

                                             tokenised  label  
x 0     house republican fret winning health care suit  0.375  
  1  rift officer resident killing persist south bronx  0.000  
  2  tyrus wong ‘bambi’ artist thwarted racial bias dy  0.000  
  3                   among death heavy toll pop music -0.100  
  4  kim jong-un say north korea preparing test lon...  0.000

In [ ]:
print(title_tb['label'].value_counts())

 0.000000    83923
 0.061983     2915
 0.040000     1744
 0.250000     1603
-0.500000     1173
             ...  
 0.038403        1
 0.025977        1
 0.040486        1
-0.148970        1
 0.058202        1
Name: label, Length: 6767, dtype: int64


In [ ]:
polar = []
count = 0

for index, row in article_tb.iterrows():
    count += 1
    print(count)

    textBlob = TextBlob(row['tokenised'])
    polar.append(textBlob.polarity * textBlob.subjectivity)

article_tb['label'] = polar

article_tb.head()

Streaming output truncated to the last 5000 lines.
137571
137572
137573
137574
137575
137576
137577
137578
137579
137580
137581
137582
137583
137584
137585
137586
137587
137588
137589
137590
137591
137592
137593
137594
137595
137596
137597
137598
137599
137600
137601
137602
137603
137604
137605
137606
137607
137608
137609
137610
137611
137612
137613
137614
137615
137616
137617
137618
137619
137620
137621
137622
137623
137624
137625
137626
137627
137628
137629
137630
137631
137632
137633
137634
137635
137636
137637
137638
137639
137640
137641
137642
137643
137644
137645
137646
137647
137648
137649
137650
137651
137652
137653
137654
137655
137656
137657
137658
137659
137660
137661
137662
137663
137664
137665
137666
137667
137668
137669
137670
137671
137672
137673
137674
137675
137676
137677
137678
137679
137680
137681
137682
137683
137684
137685
137686
137687
137688
137689
137690
137691
137692
137693
137694
137695
137696
137697
137698
137699
137700
137701
137702
137703
137704
137705
1377

article  \
x 0  WASHINGTON  —   Congressional Republicans have...   
  1  After the bullet shells get counted, the blood...   
  2  When Walt Disney’s “Bambi” opened in 1942, cri...   
  3  Death may be the great equalizer, but it isn’t...   
  4  SEOUL, South Korea  —   North Korea’s leader, ...   

                                             tokenised     label  
x 0  washington congressional republican new fear c...  0.014166  
  1  bullet shell get counted blood dry votive cand... -0.025156  
  2  walt disney’s bambi opened critic praised spar...  0.014525  
  3  death may great equalizer isn’t necessarily ev...  0.050653  
  4  seoul south korea north korea’s leader kim sai...  0.058297

In [ ]:
print(article_tb['label'].value_counts())

 0.000000    1009
 0.061983      48
 0.250000      24
 0.030000      19
 0.040000      18
             ... 
-0.005577       1
 0.052804       1
 0.088779       1
 0.026531       1
 0.015585       1
Name: label, Length: 139857, dtype: int64


In [ ]:
# save values for future uses
article_tb.to_csv("/content/drive/MyDrive/info/article_tb.csv", sep='\t')
title_tb.to_csv("/content/drive/MyDrive/info/title_tb.csv", sep='\t')

In [ ]:
# in case data needs to be uploaded
article_tb = pd.read_csv("/content/drive/MyDrive/info/article_tb.csv", sep='\t', lineterminator='\n')
title_tb = pd.read_csv("/content/drive/MyDrive/info/title_tb.csv", sep='\t', lineterminator='\n')

article_tb.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)
title_tb.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)

article_tb.head()
title_tb.head()

,title,tokenised,label
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit,0.375
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx,0.000
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy,0.000
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music,-0.100
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...,0.000


In [ ]:
a_tb_labels = article_tb['label']
t_tb_labels = title_tb['label']

a_tb_labels.to_csv("/content/drive/MyDrive/info/a_tb_labels.csv", sep='\t')
t_tb_labels.to_csv("/content/drive/MyDrive/info/t_tb_labels.csv", sep='\t')

# Labeling with Vader

In [ ]:
#title_va = df_title.copy()
#article_va = df_article.copy()

title_va = title_tt.copy()
article_va = article_tt.copy()

In [ ]:
## Function to print sentiments of the sentence.
def get_vader_scores(text):
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(text)
    comp_score = sentiment_dict['compound']
    return comp_score

In [ ]:
vader = []
count = 0

for index, row in title_va.iterrows():
    count += 1
    print(count)

    vader.append(get_vader_scores(row['tokenised']))

title_va['label'] = vader

title_va.head()

Streaming output truncated to the last 5000 lines.
137571
137572
137573
137574
137575
137576
137577
137578
137579
137580
137581
137582
137583
137584
137585
137586
137587
137588
137589
137590
137591
137592
137593
137594
137595
137596
137597
137598
137599
137600
137601
137602
137603
137604
137605
137606
137607
137608
137609
137610
137611
137612
137613
137614
137615
137616
137617
137618
137619
137620
137621
137622
137623
137624
137625
137626
137627
137628
137629
137630
137631
137632
137633
137634
137635
137636
137637
137638
137639
137640
137641
137642
137643
137644
137645
137646
137647
137648
137649
137650
137651
137652
137653
137654
137655
137656
137657
137658
137659
137660
137661
137662
137663
137664
137665
137666
137667
137668
137669
137670
137671
137672
137673
137674
137675
137676
137677
137678
137679
137680
137681
137682
137683
137684
137685
137686
137687
137688
137689
137690
137691
137692
137693
137694
137695
137696
137697
137698
137699
137700
137701
137702
137703
137704
137705
1377

title  \
x 0  House Republicans Fret About Winning Their Hea...   
  1  Rift Between Officers and Residents as Killing...   
  2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
  3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
  4  Kim Jong-un Says North Korea Is Preparing to T...   

                                             tokenised   label  
x 0     house republican fret winning health care suit  0.7650  
  1  rift officer resident killing persist south bronx -0.6597  
  2  tyrus wong ‘bambi’ artist thwarted racial bias dy -0.1280  
  3                   among death heavy toll pop music -0.5994  
  4  kim jong-un say north korea preparing test lon...  0.0000

In [ ]:
print(title_va['label'].value_counts())

 0.0000    53252
-0.3400     2558
-0.4767     2438
 0.4019     2111
-0.3818     2048
           ...  
-0.7414        1
 0.3760        1
-0.3076        1
-0.6887        1
 0.5216        1
Name: label, Length: 1072, dtype: int64


In [ ]:
title_va.to_csv("/content/drive/MyDrive/info/title_va.csv", sep='\t')

In [ ]:
a_va_list = article_va["tokenised"].to_list()
a_va_list[3824]

count = 0
for i in a_va_list:
  count+= 1

  if pd.isna(i):
    print(i)
    print('count: ' + str(count))

nan
count: 3824
nan
count: 39870
nan
count: 40127
nan
count: 42039
nan
count: 51177
nan
count: 57061
nan
count: 60578
nan
count: 60716
nan
count: 61665
nan
count: 66694
nan
count: 66706
nan
count: 66784
nan
count: 66877
nan
count: 67018
nan
count: 67144
nan
count: 67262
nan
count: 67263
nan
count: 67375
nan
count: 67381
nan
count: 67535
nan
count: 67821
nan
count: 67981
nan
count: 68133
nan
count: 69248
nan
count: 70100
nan
count: 70149
nan
count: 71195
nan
count: 71428
nan
count: 72931
nan
count: 73021
nan
count: 73112
nan
count: 73288
nan
count: 73307
nan
count: 73347
nan
count: 73369
nan
count: 74035
nan
count: 98852


In [ ]:
# runs 2 hrs
vader = []
count = 0

for index, row in article_va.iterrows():
    count += 1

    if count % 1000 == 0:
      print(count)

    if not pd.isna(row['tokenised']):
      vader.append(get_vader_scores(row['tokenised']))
    else:
      print("This value is NaN")
      vader.append(-1)

article_va['label'] = vader

article_va.head()

1000
2000
3000
This value is NaN
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
This value is NaN
40000
This value is NaN
41000
42000
This value is NaN
43000
44000
45000
46000
47000
48000
49000
50000
51000
This value is NaN
52000
53000
54000
55000
56000
57000
This value is NaN
58000
59000
60000
This value is NaN
This value is NaN
61000
This value is NaN
62000
63000
64000
65000
66000
This value is NaN
This value is NaN
This value is NaN
This value is NaN
67000
This value is NaN
This value is NaN
This value is NaN
This value is NaN
This value is NaN
This value is NaN
This value is NaN
This value is NaN
This value is NaN
68000
This value is NaN
69000
This value is NaN
70000
This value is NaN
This value is NaN
71000
This value is NaN
This value is NaN
72000
This value is NaN
73000
This value is NaN
This value is NaN
This value is

,article,tokenised,label
0,WASHINGTON — Congressional Republicans have...,washington congressional republican new fear c...,0.2982
1,"After the bullet shells get counted, the blood...",bullet shell get counted blood dry votive cand...,-0.9999
2,"When Walt Disney’s “Bambi” opened in 1942, cri...",walt disney’s bambi opened critic praised spar...,0.9885
3,"Death may be the great equalizer, but it isn’t...",death may great equalizer isn’t necessarily ev...,-0.8863
4,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea’s leader kim sai...,0.9789


In [ ]:
print(article_va['label'].value_counts())

-0.0000    868
 0.9987    477
 0.9978    446
 0.9988    446
 0.9982    444
          ... 
 0.5507      1
-0.5468      1
 0.4320      1
 0.1021      1
-0.7395      1
Name: label, Length: 9452, dtype: int64


In [ ]:
# save values for future uses
article_va.to_csv("/content/drive/MyDrive/info/article_va.csv", sep='\t')

In [ ]:
# in case data needs to be uploaded
article_va = pd.read_csv("/content/drive/MyDrive/info/article_va.csv", sep='\t', lineterminator='\n')
title_va = pd.read_csv("/content/drive/MyDrive/info/title_va.csv", sep='\t', lineterminator='\n')

article_va.drop(columns = ["Unnamed: 0"], inplace = True)
title_va.drop(columns = ["Unnamed: 0", "Unnamed: 1"], inplace = True)

article_va.head()
title_va.head()

,title,tokenised,label
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit,0.7650
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx,-0.6597
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy,-0.1280
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music,-0.5994
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...,0.0000


In [ ]:
a_va_labels = article_va['label']
t_va_labels = title_va['label']

a_va_labels.to_csv("/content/drive/MyDrive/info/a_va_labels.csv", sep='\t')
t_va_labels.to_csv("/content/drive/MyDrive/info/t_va_labels.csv", sep='\t')

# All labels

In [ ]:
#title_ll = df_title.copy()
#article_ll = df_article.copy()

title_ll = title_tt.copy()
article_ll = article_tt.copy()

In [ ]:
# get title label columns and create a list
t_sn_column = pd.read_csv("/content/drive/MyDrive/info/t_sn_labels.csv", sep='\t', lineterminator='\n')
t_sn_list = t_sn_column['label'].to_list()

t_di_column = pd.read_csv("/content/drive/MyDrive/info/t_di_labels.csv", sep='\t', lineterminator='\n')
t_di_list = t_di_column['label'].to_list()

t_tb_column = pd.read_csv("/content/drive/MyDrive/info/t_tb_labels.csv", sep='\t', lineterminator='\n')
t_tb_list = t_tb_column['label'].to_list()

t_va_column = pd.read_csv("/content/drive/MyDrive/info/t_va_labels.csv", sep='\t', lineterminator='\n')
t_va_list = t_va_column['label'].to_list()

In [ ]:
# get article label columns and create a list
a_sn_column = pd.read_csv("/content/drive/MyDrive/info/a_sn_labels.csv", sep='\t', lineterminator='\n')
a_sn_list = a_sn_column['label'].to_list()

a_di_column = pd.read_csv("/content/drive/MyDrive/info/a_di_labels.csv", sep='\t', lineterminator='\n')
a_di_list = a_di_column['label'].to_list()

a_tb_column = pd.read_csv("/content/drive/MyDrive/info/a_tb_labels.csv", sep='\t', lineterminator='\n')
a_tb_list = a_tb_column['label'].to_list()

a_va_column = pd.read_csv("/content/drive/MyDrive/info/a_va_labels.csv", sep='\t', lineterminator='\n')
a_va_list = a_va_column['label'].to_list()

In [ ]:
t_agg_labels = []
a_agg_labels = []

di_weight = 0.85
tb_weight = 0.1
va_weight = 0.05

weight = [di_weight, tb_weight, va_weight]

for i in range(len(t_sn_list)):
    value = np.sum([weight[0]*t_di_list[i], weight[1]*t_tb_list[i], weight[2]*t_va_list[i]])
    t_agg_labels.append(value)


for i in range(len(a_sn_list)):
    value = np.sum([weight[0]*a_di_list[i], weight[1]*a_tb_list[i], weight[2]*a_va_list[i]])
    a_agg_labels.append(value)


t_class_list = []
for i in t_agg_labels:
    if i >= 0.1:
        t_class_list.append(1) #positive
    elif i <= -0.1:
        t_class_list.append(0) #negative
    else:
        t_class_list.append(-1) #neutral

a_class_list = []
for i in a_agg_labels:
    if i >= 0.1:
        a_class_list.append(1) #positive
    elif i <= -0.1:
        a_class_list.append(0) #negative
    else:
        a_class_list.append(-1) #neutral

In [ ]:
a_class_list.count(1)

60508

In [ ]:
a_class_list.count(0)

44636

In [ ]:
title_ll.head()

,title,tokenised
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...


In [ ]:
# add title labels to an overall dataframe
title_ll = pd.concat([title_ll, t_sn_column['label']], axis=1)
title_ll.rename(columns = {'label' : 'label_sn'}, inplace = True)

title_ll = pd.concat([title_ll, t_di_column['label']], axis=1)
title_ll.rename(columns = {'label' : 'label_di'}, inplace = True)

title_ll = pd.concat([title_ll, t_tb_column['label']], axis=1)
title_ll.rename(columns = {'label' : 'label_tb'}, inplace = True)

title_ll = pd.concat([title_ll, t_va_column['label']], axis=1)
title_ll.rename(columns = {'label' : 'label_va'}, inplace = True)

title_ll['agg_label'] = t_class_list

title_ll.head()

,title,tokenised,label_sn,label_di,label_tb,label_va,agg_label
0,House Republicans Fret About Winning Their Hea...,house republican fret winning health care suit,1,0.571429,0.375,0.7650,1
1,Rift Between Officers and Residents as Killing...,rift officer resident killing persist south bronx,-1,-0.285714,0.000,-0.6597,0
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",tyrus wong ‘bambi’ artist thwarted racial bias dy,0,-0.125000,0.000,-0.1280,0
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",among death heavy toll pop music,1,-0.333333,-0.100,-0.5994,0
4,Kim Jong-un Says North Korea Is Preparing to T...,kim jong-un say north korea preparing test lon...,-1,-0.111111,0.000,0.0000,-1


In [ ]:
# add article labels to an overall dataframe
article_ll = pd.concat([article_ll, a_sn_column['label']], axis=1)
article_ll.rename(columns = {'label' : 'label_sn'}, inplace = True)

article_ll = pd.concat([article_ll, a_di_column['label']], axis=1)
article_ll.rename(columns = {'label' : 'label_di'}, inplace = True)

article_ll = pd.concat([article_ll, a_tb_column['label']], axis=1)
article_ll.rename(columns = {'label' : 'label_tb'}, inplace = True)

article_ll = pd.concat([article_ll, a_va_column['label']], axis=1)
article_ll.rename(columns = {'label' : 'label_va'}, inplace = True)

article_ll['agg_label'] = a_class_list

article_ll.head()

,article,tokenised,label_sn,label_di,label_tb,label_va,agg_label
0,WASHINGTON — Congressional Republicans have...,washington congressional republican new fear c...,1,0.152174,0.014166,0.2982,1
1,"After the bullet shells get counted, the blood...",bullet shell get counted blood dry votive cand...,1,-0.124178,-0.025156,-0.9999,0
2,"When Walt Disney’s “Bambi” opened in 1942, cri...",walt disney’s bambi opened critic praised spar...,1,0.078089,0.014525,0.9885,1
3,"Death may be the great equalizer, but it isn’t...",death may great equalizer isn’t necessarily ev...,1,-0.089478,0.050653,-0.8863,0
4,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea’s leader kim sai...,1,-0.070588,0.058297,0.9789,-1


In [ ]:
# save values for future uses
article_ll.to_csv("/content/drive/MyDrive/info/article_df_final.csv", sep='\t')
title_ll.to_csv("/content/drive/MyDrive/info/title_df_final.csv", sep='\t')

# Loading labelled data

In [ ]:
df_article = pd.read_csv("/content/drive/MyDrive/info/article_df_final.csv", sep='\t', lineterminator='\n')
df_title = pd.read_csv("/content/drive/MyDrive/info/title_df_final.csv", sep='\t', lineterminator='\n')

In [ ]:
df_article.fillna('', inplace=True)
df_article.drop(columns = ['Unnamed: 0'], inplace=True)
df_title.fillna('', inplace=True)
df_title.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
df_article.head()

,article,tokenised,label_sn,label_di,label_tb,label_va,agg_label
0,WASHINGTON — Congressional Republicans have...,washington congressional republican new fear c...,1,0.152174,0.014166,0.2982,1
1,"After the bullet shells get counted, the blood...",bullet shell get counted blood dry votive cand...,1,-0.124178,-0.025156,-0.9999,0
2,"When Walt Disney’s “Bambi” opened in 1942, cri...",walt disney’s bambi opened critic praised spar...,1,0.078089,0.014525,0.9885,1
3,"Death may be the great equalizer, but it isn’t...",death may great equalizer isn’t necessarily ev...,1,-0.089478,0.050653,-0.8863,0
4,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea’s leader kim sai...,1,-0.070588,0.058297,0.9789,-1
